In [1]:
import requests
from bs4 import BeautifulSoup

In [21]:
url = 'https://www.livemint.com/economy/budget-2024-25-key-highlights-live-updates-interim-budget-agriculture-infra-fiscal-deficit-nirmala-sitharaman-11706695416199.html'

In [22]:
response = requests.get(url)
soup = BeautifulSoup(response.content,'html.parser')
text = soup.get_text()

In [23]:
text

'\n\n\n\n          \n\n\n \n\n\n\n\nBudget 2024 Key Highlights: India to cut aid to the Maldives by 22% | Mint\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n  \n\n\n\n\n\n  \n\n\n\n\n \n\n\n\n\n\n\n\n          Explore    Sign in e-paper Subscribe Thursday, 8 February 2024              Stocks       Mutual Funds       News                     Home Budget-2024 News Markets Premium Money Mutual Fund Industry Companies Technology Web Stories Opinion Videos      All Companies  Technology Markets Money Mutual Funds Insurance Auto  Industry  Personal Finance        Hello User  Sign in      Sign Out     My Account My Account Subscribe  My Watchlist   Newsletters   Notifications      My Reads   For You   View Less -  View More +   Data Insights   Market Dashboard   Bullion      Gold   Silver     Fuel      Petrol   Diesel     Commodities      Gold   GoldM   Aluminum   Menthaoil   Silver   SilverMIC   GoldPetal   Natural Gas   Copper   Zinc   SilverM   CrudeOil   GoldGinuea   Lead

In [5]:
from transformers import BertTokenizer,BertForSequenceClassification

In [6]:
from transformers import pipeline

In [24]:
tokenizer = BertTokenizer.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis")
tokens = tokenizer(text, return_tensors="pt")

Token indices sequence length is longer than the specified maximum sequence length for this model (11404 > 512). Running this sequence through the model will result in indexing errors


In [14]:
model = BertForSequenceClassification.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis", num_labels=3)

In [25]:
# Tokenize the text using the FinancialBERT tokenizer with truncation
tokens = tokenizer(text, truncation=True, return_tensors="pt", max_length=512)

# Ensure the input tensor size matches the expected size
tokens['input_ids'] = tokens['input_ids'][:, :512]

# Forward pass
outputs = model(**tokens)
logits = outputs.logits

In [26]:
import torch
import numpy as np

# Assuming 'logits' is the tensor obtained from the model
logits = outputs.logits

# Convert logits to probabilities using softmax
probs = torch.nn.functional.softmax(logits, dim=-1)

# Get the predicted sentiment label
predicted_label = torch.argmax(probs, dim=-1).item()

# Optionally, get the confidence score for the predicted label
confidence_score = probs[0, predicted_label].item()

# Print the results
print("Predicted Sentiment Label:", predicted_label)
print("Confidence Score:", confidence_score)


Predicted Sentiment Label: 1
Confidence Score: 0.9970109462738037


In [27]:
import torch
import numpy as np

# Split the text into lines
lines = text.split('\n')

# Initialize an empty list to store the sentiment results for each line
sentiment_results = []

# Perform sentiment analysis for each line
for line in lines:
    # Skip empty lines
    if not line.strip():
        continue

    # Tokenize the line using the FinancialBERT tokenizer
    tokens = tokenizer(line, truncation=True, return_tensors="pt", max_length=512)
    tokens['input_ids'] = tokens['input_ids'][:, :512]

    # Forward pass
    outputs = model(**tokens)
    logits = outputs.logits

    # Convert logits to probabilities using softmax
    probs = torch.nn.functional.softmax(logits, dim=-1)

    # Get the predicted sentiment label
    predicted_label = torch.argmax(probs, dim=-1).item()

    # Optionally, get the confidence score for the predicted label
    confidence_score = probs[0, predicted_label].item()

    # Append the sentiment result for this line to the list
    sentiment_results.append({
        'line': line,
        'predicted_label': predicted_label,
        'confidence_score': confidence_score
    })

# Print the sentiment results for each line
for result in sentiment_results:
    print(f"Line: {result['line']}")
    print(f"Predicted Sentiment Label: {result['predicted_label']}")
    print(f"Confidence Score: {result['confidence_score']}")
    print("\n")


Line: Budget 2024 Key Highlights: India to cut aid to the Maldives by 22% | Mint
Predicted Sentiment Label: 0
Confidence Score: 0.9918386340141296


Line:           Explore    Sign in e-paper Subscribe Thursday, 8 February 2024              Stocks       Mutual Funds       News                     Home Budget-2024 News Markets Premium Money Mutual Fund Industry Companies Technology Web Stories Opinion Videos      All Companies  Technology Markets Money Mutual Funds Insurance Auto  Industry  Personal Finance        Hello User  Sign in      Sign Out     My Account My Account Subscribe  My Watchlist   Newsletters   Notifications      My Reads   For You   View Less -  View More +   Data Insights   Market Dashboard   Bullion      Gold   Silver     Fuel      Petrol   Diesel     Commodities      Gold   GoldM   Aluminum   Menthaoil   Silver   SilverMIC   GoldPetal   Natural Gas   Copper   Zinc   SilverM   CrudeOil   GoldGinuea   Lead        CryptoCurrencies   View Less -  View More +   Top Sect

In [28]:
import requests
from bs4 import BeautifulSoup
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Function to perform sentiment analysis on a given sentence
def analyze_sentiment(sentence, model, tokenizer):
    tokens = tokenizer(sentence, truncation=True, return_tensors="pt", max_length=512)
    tokens['input_ids'] = tokens['input_ids'][:, :512]

    outputs = model(**tokens)
    logits = outputs.logits
    probs = torch.nn.functional.softmax(logits, dim=-1)

    predicted_label = torch.argmax(probs, dim=-1).item()
    sentiment_text = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}[predicted_label]
    confidence_score = probs[0, predicted_label].item()

    return {'label': sentiment_text, 'score': confidence_score}

# Load the FinancialBERT model and tokenizer
model = BertForSequenceClassification.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis", num_labels=3)
tokenizer = BertTokenizer.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis")

# Specify the URL
url = 'https://www.livemint.com/economy/budget-2024-25-key-highlights-live-updates-interim-budget-agriculture-infra-fiscal-deficit-nirmala-sitharaman-11706695416199.html'

# Make a request to the URL
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Extract text from all paragraphs
paragraphs = soup.find_all('p')
text = ' '.join([paragraph.get_text() for paragraph in paragraphs])

# Split the text into sentences
sentences = text.split('.')

# Initialize an empty list to store the sentiment results for each sentence
sentiment_results = []

# Perform sentiment analysis for each sentence
for i, sentence in enumerate(sentences, start=1):  # Starting index from 1
    # Skip empty sentences
    if not sentence.strip():
        continue

    # Analyze sentiment for the current sentence
    sentiment_result = analyze_sentiment(sentence, model, tokenizer)

    # Append the sentiment result for this sentence to the list
    sentiment_results.append({
        'sentence_number': i,
        'sentence': sentence,
        'sentiment': sentiment_result
    })

# Print the sentiment results for each sentence
for result in sentiment_results:
    print(f"Sentence {result['sentence_number']}: {result['sentence']}")
    print(f"Sentiment: {result['sentiment']}")
    print("\n")


Sentence 1: Budget 2024 Key Highlights LIVE Updates: Union Finance Minister Nirmala Sitharaman presented her sixth Budget on February 1
Sentiment: {'label': 'Neutral', 'score': 0.9997792840003967}


Sentence 2:  This was an interim budget ahead of the general elections later this year
Sentiment: {'label': 'Neutral', 'score': 0.9995469450950623}


Sentence 3:  The Interim Budget 2024 was focused on youth and women empowerment, while maintaining fiscal consolidation and continuing capex
Sentiment: {'label': 'Neutral', 'score': 0.9998157620429993}


Sentence 4:  FM Sitharaman lowered down FY25 fiscal deficit target to 5
Sentiment: {'label': 'Negative', 'score': 0.9303200244903564}


Sentence 5: 1% of the GDP
Sentiment: {'label': 'Neutral', 'score': 0.9996143579483032}


Sentence 6:  There were no changes made to the direct tax and indirect tax rates
Sentiment: {'label': 'Neutral', 'score': 0.9995985627174377}


Sentence 7:  Read the latest news on Budget 2024 with our live coverage on the

In [29]:
from transformers import BartForConditionalGeneration, BartTokenizer

# Load the Bart model and tokenizer for summarization
model_summarization = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
tokenizer_summarization = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

# Tokenize and summarize the text
input_ids = tokenizer_summarization.encode(text, return_tensors="pt", max_length=1024, truncation=True)
summary_ids = model_summarization.generate(input_ids, max_length=150, length_penalty=2.0, num_beams=4, early_stopping=True)
summary = tokenizer_summarization.decode(summary_ids[0], skip_special_tokens=True)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Summary: Budget 2024 Key Highlights LIVE Updates: Union Finance Minister Nirmala Sitharaman presented her sixth Budget on February 1. This was an interim budget ahead of the general elections later this year. The Interim Budget 2024 was focused on youth and women empowerment, while maintaining fiscal consolidation and continuing capex.


In [30]:
# Print the summarized text
print("Summary:", summary)

Summary: Budget 2024 Key Highlights LIVE Updates: Union Finance Minister Nirmala Sitharaman presented her sixth Budget on February 1. This was an interim budget ahead of the general elections later this year. The Interim Budget 2024 was focused on youth and women empowerment, while maintaining fiscal consolidation and continuing capex.
